In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h') # - timedelta(hours=1)
current_date = current_date.tz_localize('UTC')
print(f'{current_date=}')

In [ ]:
import src.config as config

In [ ]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

In [ ]:
features.head(10)

In [ ]:
features.shape

In [ ]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

In [ ]:
predictions['pickup_hour'] = current_date
predictions

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})